In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
path = 'csv/glassdoor_data_scientist_canada_221118_150.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Junior Data Scientist,CA$74K - CA$90K (Glassdoor est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,-1,Mississauga,1001 to 5000 Employees,-1,Company - Private,Animal Production,Agriculture,$25 to $100 million (USD)
1,-1,CA$55K - CA$65K (Glassdoor est.),-1,3.5,-1,-1,-1,1001 to 5000 Employees,2005,Company - Private,Taxi & Car Services,Transportation & Logistics,$5 to $25 million (USD)
2,Data Scientist,CA$85K - CA$120K (Glassdoor est.),Referred applicants should not apply directly ...,3.6,Loblaw Companies Limited\n3.6,-1,Brampton,10000+ Employees,-1,Company - Public,Vehicle Dealers,Retail & Wholesale,$10+ billion (USD)
3,Data Scientist – Revenue Management,CA$84K - CA$96K (Glassdoor est.),Powered by water... and by people like you\n\n...,-1.0,BC Hydro\n3.8,-1,Vancouver,1001 to 5000 Employees,-1,Self-employed,Energy & Utilities,"Energy, Mining & Utilities",Unknown / Non-Applicable
4,Data Scientist,CA$55K - CA$122K (Glassdoor est.),Join our Winning Team as a Data Scientist\nWhe...,4.5,Carfax\n4.5,-1,London,1001 to 5000 Employees,1984,Company - Public,Internet & Web Services,Information Technology,Unknown / Non-Applicable


In [5]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Division               int64
Location              object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [6]:
def clean(path):
    
    df = pd.read_csv(path)
    
    # Parse Company Name
    # Clean the company name as Glassdoor attaches the comapny rating after the name
    df['Company'] = df['Company Name'].apply(lambda x: x.split('\n')[0])
    
    # Indicator column for whether the salary estimate is per hour pay
    df['Hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
    
    # Indicator column for whether salary estimate is posted by the employer
    df['Employer Estimate'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

    # Remove entries with blank salary estimate
    # Remove entries where job title, job desc, and company name are missing
    df = df[df['Salary Estimate'] != '-1']
    blank_indecies = df[ (df['Job Title'] == '-1') & (df['Job Description'] == '-1') & (df['Company Name'] == '-1') ].index
    df.drop(blank_indecies , inplace=True)
    
    # Clean the salary estimate tab of any symbols and excess words
    # Desired format: eg: 40-50
    salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
    currency = df['Salary Estimate'].apply(lambda x: 'CAD' if 'CA' in x else None)
    no_signs = salary.apply(lambda x: re.sub('[^0-9,\-,-.]','', x))
    
    df['Salary Currency'] = currency
    
    # Split the cleaned salary range into min, max, and avg numeric values
    df['min_salary'] = no_signs.apply(lambda x: float(x.split('-')[0]) if '-' in x else float(x))
    df['max_salary'] = no_signs.apply(lambda x: float(x.split('-')[1]) if '-' in x else float(x))
    df['avg_salary'] = (df.min_salary+df.max_salary)/2
    
    
    # Parsing of job description for keywords(python, etc.)
    # Creating indicator columns for whether the keywords are present
    
        ## Languages
    #python
    python_keywords = ['python', 'py']
    df['Python'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in python_keywords) else 0)

    #R
    R_keywords = [' r ', 'r language', 'r-language']
    df['R'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in R_keywords) else 0)
    df.R.value_counts()
    
    #SQL
    SQL_keywords = ['sql']
    df['SQL'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in SQL_keywords) else 0)
    df.SQL.value_counts()
    
    #Java
    Java_keywords = ['java']
    df['Java'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in Java_keywords) else 0)
    df.Java.value_counts()

        ## Technologies
    #r studio 
    Rs_keywords = ['r studio', 'r-studio']
    df['R_Studio'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in Rs_keywords) else 0)
    df.R_Studio.value_counts()
    
    #spark 
    spark_keywords = ['spark', 'apache spark']
    df['spark'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in spark_keywords) else 0)
    df.spark.value_counts()

    #aws 
    aws_keywords = ['aws', 'amazon web service', 'amazon web services']
    df['aws'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in aws_keywords) else 0)
    df.aws.value_counts()

    #excel
    excel_keywords = ['excel', 'xlsx', 'microsoft']
    df['excel'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in excel_keywords) else 0)
    df.excel.value_counts()
    
    #Google BigQuery
    BQ_keywords = ['big query', 'bigquery', 'google query', 'google-bigquery', 'google-bigquery']
    df['BigQuery'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in BQ_keywords) else 0)
    df.BigQuery.value_counts()
    
    #SQL Server
    SS_keywords = ['sqlserver', 'sql server', 'mircrosoft-sqlserver']
    df['SQL_Server'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in SS_keywords) else 0)
    df.SQL_Server.value_counts()
    
    #Docker
    Docker_keywords = ['docker']
    df['Docker'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in Docker_keywords) else 0)
    df.Docker.value_counts()
    
    #Jira
    Jira_keywords = ['jira']
    df['Jira'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in Jira_keywords) else 0)
    df.Jira.value_counts()
    
        ## Work Style
    #hybrid
    hybrid_keywords = ['hybrid']
    df['hybrid'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in hybrid_keywords) else 0)
    df.hybrid.value_counts()
    
    #remote
    remote_keywords = ['remote']
    df['remote'] = df['Job Description'].apply(lambda x: 1 if any(keyword in x.lower() for keyword in remote_keywords) else 0)
    df.remote.value_counts()

    df.columns

    # Attach 'cleaned' to filename, save and export
    new_path = path[:-4] + '_cleaned.csv'
    print('new path: ' + new_path)
    
    df_out = df#.drop(['Unnamed: 0'], axis =1)

    df_out.to_csv(new_path,index = False)
    return

In [7]:
clean(path)
df_cleaned = pd.read_csv(path[:-4] + '_cleaned.csv')
df_cleaned.head(50)

new path: csv/glassdoor_data_scientist_canada_221118_150_cleaned.csv


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,...,R_Studio,spark,aws,excel,BigQuery,SQL_Server,Docker,Jira,hybrid,remote
0,Junior Data Scientist,CA$74K - CA$90K (Glassdoor est.),We are looking for our next Junior Data Scient...,3.9,M&M Food Market\n3.9,-1,Mississauga,1001 to 5000 Employees,-1,Company - Private,...,0,1,0,1,0,0,0,0,0,0
1,Data Scientist,CA$85K - CA$120K (Glassdoor est.),Referred applicants should not apply directly ...,3.6,Loblaw Companies Limited\n3.6,-1,Brampton,10000+ Employees,-1,Company - Public,...,0,1,0,1,0,0,0,1,1,1
2,Data Scientist – Revenue Management,CA$84K - CA$96K (Glassdoor est.),Powered by water... and by people like you\n\n...,-1.0,BC Hydro\n3.8,-1,Vancouver,1001 to 5000 Employees,-1,Self-employed,...,0,0,0,0,0,0,0,0,1,1
3,Data Scientist,CA$55K - CA$122K (Glassdoor est.),Join our Winning Team as a Data Scientist\nWhe...,4.5,Carfax\n4.5,-1,London,1001 to 5000 Employees,1984,Company - Public,...,0,1,0,0,0,0,0,0,0,0
4,Analytics Data Scientist,CA$65K - CA$100K (Employer est.),BrainFinance is a leading financial technology...,-1.0,Brain Finance,-1,Montreal,Unknown,-1,Company - Private,...,0,0,0,1,0,0,0,0,0,0
5,"Team Lead, Decision Scientist for Cancer Scree...",CA$54K - CA$85K (Glassdoor est.),Want to make a difference in your career? Cons...,3.0,Ontario Health\n3.0,-1,Toronto,10000+ Employees,-1,Company - Public,...,0,0,0,1,0,0,0,0,1,0
6,Junior Software Developer: Data Science,CA$55K - CA$75K (Employer est.),About Us\nCopperstone helps mining companies m...,5.0,Copperstone Technologies Ltd.\n5.0,-1,Edmonton,1 to 50 Employees,-1,Company - Private,...,0,1,0,0,1,0,0,0,0,1
7,Data Scientist (Deep Learning),CA$135K - CA$150K (Employer est.),Le Data Scientist (Deep Learning) fera partie ...,5.0,DELAN - Chasseurs de Talents en TI\n5.0,-1,Montreal,1 to 50 Employees,1997,Company - Private,...,0,1,0,0,0,0,0,0,1,0
8,Data Scientist,CA$134K - CA$191K (Glassdoor est.),"As a Data Scientist, you’ll bring a deep under...",-1.0,Maarut Inc,-1,Calgary,51 to 200 Employees,-1,Private Practice / Firm,...,0,0,0,1,0,0,0,0,1,1
9,Data Scientist Lead,CA$71K - CA$125K (Glassdoor est.),Job ID #349220\nData Scientist Lead\nOn behalf...,3.9,Procom\n3.9,-1,Vancouver,5001 to 10000 Employees,1978,Company - Private,...,0,1,0,1,0,0,0,0,0,0


In [8]:
# Columns 0-12: Scraped data from Glassdoor
# Columns 13-34: Custom non-scrapped columns
# 13: parsed company name
# 14: Hourly indicator
# 15: Employer provided salary indicator
# 16: currency
# 17-19: Salary info
# 20-34: Job description keyword indicators
df_cleaned.iloc[0]

Job Title                                        Junior Data Scientist
Salary Estimate                       CA$74K - CA$90K (Glassdoor est.)
Job Description      We are looking for our next Junior Data Scient...
Rating                                                             3.9
Company Name                                      M&M Food Market\n3.9
Division                                                            -1
Location                                                   Mississauga
Size                                            1001 to 5000 Employees
Founded                                                             -1
Type of ownership                                    Company - Private
Industry                                             Animal Production
Sector                                                     Agriculture
Revenue                                      $25 to $100 million (USD)
Company                                                M&M Food Market
Hourly

In [9]:
df_cleaned.iloc[:, 20:].sum()

Python        73
R              8
SQL           62
Java          16
R_Studio       0
spark         19
aws           12
excel         56
BigQuery       8
SQL_Server     2
Docker         4
Jira           3
hybrid        30
remote        36
dtype: int64

In [10]:
df_cleaned.iloc[40, 2]

'How to Apply\nE-mail your resume and cover letter to rsaad [at] innovatebc [dot] ca with the subject line “Data + Policy Analyst” by November 16, 2022.\nCompensation: $55,000 to $65,000 per annum\nThis is a temporary, 1-year contract with the possibility of extension.\nAbout Innovate BC\nInnovate BC helps foster innovation in B.C. so that British Columbians in all regions of the province can benefit from a thriving, sustainable, and inclusive innovation economy.\u202fA Crown Agency of British Columbia, Innovate BC funds and delivers programs that support the growth of the B.C. economy by helping companies start and scale, train talent that meets labour market needs, and encourage technology development, commercialization, and adoption.\nAbout the Team\nThe Data and Policy Department works to strengthen Innovate BC’s position as a central data and resource hub for the technology and innovation ecosystem in British Columbia. This dynamic team supports the corporate goals and objectives 